### Overview

This notebook measures the speed between

 * regular python function
 * numba function
 * numba with inline numba functions

We would use a simple tradebook example to test this.

In this example, we would be BUY a instrument when the price goes above the high price of a previous lookback period or SELL the instrument when the price goes below the low price

In [18]:
import yfinance as yf
import numpy as np
from numba import njit

msft = yf.Ticker("MSFT")

# get all stock info
msft.info

# get historical market data
df = msft.history(period="max")
df.columns = [x.lower() for x in df.columns]
df.head()

,open,high,low,close,volume,dividends,stock splits
Date,,,,,,,
1986-03-13 00:00:00-05:00,0.054594,0.062623,0.054594,0.059946,1031788800,0.0,0.0
1986-03-14 00:00:00-05:00,0.059946,0.063158,0.059946,0.062087,308160000,0.0,0.0
1986-03-17 00:00:00-05:00,0.062087,0.063694,0.062087,0.063158,133171200,0.0,0.0
1986-03-18 00:00:00-05:00,0.063158,0.063694,0.061017,0.061552,67766400,0.0,0.0
1986-03-19 00:00:00-05:00,0.061552,0.062087,0.059947,0.060482,47894400,0.0,0.0


In [21]:
def tradebook_long(close, date):
    """
    long only tradebook
    """
    length = len(close)
    side = 0
    entry_price = exit_price = 0
    entry_time = exit_time = date[0]
    for i in range(1, length):
        if side == 0 and close[i] > close[i - 1]:
            side = 1
            entry_price = close[i]
            entry_time = date[i]
    exit_price = close[-1]
    exit_time = date[-1]

    return side, entry_price, entry_time, exit_price, exit_time


def tradebook_short(close, date):
    """
    short only tradebook
    """
    length = len(close)
    side = 0
    entry_price = exit_price = 0
    entry_time = exit_time = date[0]
    for i in range(1, length):
        if side == 0 and close[i] < close[i - 1]:
            side = -1
            entry_price = close[i]
            entry_time = date[i]
    exit_price = close[-1]
    exit_time = date[-1]

    return side, entry_price, entry_time, exit_price, exit_time


def tradebook(close, date, start: int = 300):
    """
    both long and short tradebook
    """
    length = len(close)
    side = 0
    entry_price = exit_price = 0
    entry_time = exit_time = date[0]
    ref_high = np.max(close[:start])
    ref_low = np.min(close[:start])
    for i in range(start, length):
        if side == 0:
            if close[i] > ref_high:
                side = 1
                entry_price = close[i]
                entry_time = date[i]
            elif close[i] < ref_low:
                side = -1
                entry_price = close[i]
                entry_time = date[i]
    exit_price = close[i]
    exit_time = date[i]
    return side, entry_price, entry_time, exit_price, exit_time


# create the numba functions
ntradebook_long = njit(tradebook_long)
ntradebook_short = njit(tradebook_short)
ntradebook = njit(tradebook)

In [27]:
# Numba inline function
@njit
def tradebook_inline(close, date, start: int = 300):
    length = len(close)
    ref_high = np.max(close[:start])
    ref_low = np.min(close[:start])
    for i in range(start, length):
        if close[i] > ref_high:
            return ntradebook_long(close[i - 1 :], date[i - 1 :])
        elif close[i] < ref_low:
            return ntradebook_short(close[i - 1 :], date[i - 1 :])


tradebook_inline(df.close.values, df.index.values)

(1,
 0.27404162287712097,
 numpy.datetime64('1987-09-28T04:00:00.000000000'),
 417.4599914550781,
 numpy.datetime64('2024-10-09T04:00:00.000000000'))

In [28]:
# Print values to see if they are right and also to run the njit function once
print(tradebook_long(df.close.values, df.index.values))
print(tradebook_short(df.close.values, df.index.values))
print(tradebook(df.close.values, df.index.values))
print(ntradebook_long(df.close.values, df.index.values))
print(ntradebook_short(df.close.values, df.index.values))
print(ntradebook(df.close.values, df.index.values))
print(tradebook_inline(df.close.values, df.index.values))

(1, 0.06208731234073639, numpy.datetime64('1986-03-14T05:00:00.000000000'), 417.4599914550781, numpy.datetime64('2024-10-09T04:00:00.000000000'))
(-1, 0.06155211851000786, numpy.datetime64('1986-03-18T05:00:00.000000000'), 417.4599914550781, numpy.datetime64('2024-10-09T04:00:00.000000000'))
(1, 0.27404162287712097, numpy.datetime64('1987-09-28T04:00:00.000000000'), 417.4599914550781, numpy.datetime64('2024-10-09T04:00:00.000000000'))
(1, 0.06208731234073639, numpy.datetime64('1986-03-14T05:00:00.000000000'), 417.4599914550781, numpy.datetime64('2024-10-09T04:00:00.000000000'))
(-1, 0.06155211851000786, numpy.datetime64('1986-03-18T05:00:00.000000000'), 417.4599914550781, numpy.datetime64('2024-10-09T04:00:00.000000000'))
(1, 0.27404162287712097, numpy.datetime64('1987-09-28T04:00:00.000000000'), 417.4599914550781, numpy.datetime64('2024-10-09T04:00:00.000000000'))
(1, 0.27404162287712097, numpy.datetime64('1987-09-28T04:00:00.000000000'), 417.4599914550781, numpy.datetime64('2024-10-0

In [30]:
# %timeit values to see if they are right and also to run the njit function once
%timeit tradebook_long(df.close.values, df.index.values)
%timeit tradebook_short(df.close.values, df.index.values)
%timeit tradebook(df.close.values, df.index.values)
%timeit ntradebook_long(df.close.values, df.index.values)
%timeit ntradebook_short(df.close.values, df.index.values)
%timeit ntradebook(df.close.values, df.index.values)
%timeit tradebook_inline(df.close.values, df.index.values)

490 µs ± 6.92 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
493 µs ± 3.32 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
526 µs ± 3.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
19.1 µs ± 22.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
19.2 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
20.4 µs ± 114 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
20.3 µs ± 79.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
